<a href="https://colab.research.google.com/github/johnkidsm/FT-scripts/blob/main/fine_tuning_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4 -q
!apt update -q
!apt install -y ffmpeg -q

usage: add-apt-repository [-h] [-d] [-r] [-s] [-c COMPONENT] [-p POCKET] [-y] [-n] [-l]
                          [--dry-run] [-L] [-P PPA] [-C CLOUD] [-U URI]
                          [-S SOURCESLIST [SOURCESLIST ...]]
                          [line ...]
add-apt-repository: error: unrecognized arguments: -q
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,074 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ub

In [3]:
!pip install -q datasets>=2.6.1
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q librosa
!pip install -q evaluate>=0.30
!pip install -q jiwer
!pip install -q gradio
!pip install -q bitsandbytes datasets accelerate loralib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 120.6 MB/s eta 0:0

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

model_name_or_path = "openai/whisper-small"
language = "English"
language_abbr = "en"
task = "transcribe"
dataset_name = "Trelis/llm-lingo"

org = "johnkidsm"
trained_adapter_name = "whisper-small-llm-ling-adapters"
trained_model_name = "whisper-small-llm-ling"

trained_adapter_repo = org + "/" + trained_adapter_name
trained_model_repo = org + "/" + trained_model_name





In [5]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()

dataset["train"] = load_dataset(dataset_name, split="train")
dataset["validation"] = load_dataset(dataset_name, split="validation")

print(dataset)

README.md:   0%|          | 0.00/500 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'start_time', 'end_time'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['audio', 'text', 'start_time', 'end_time'],
        num_rows: 5
    })
})


In [6]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [7]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [8]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

In [9]:
print(dataset["train"][1])

{'audio': {'path': 'segment_1.mp3', 'array': array([-0.0324074 , -0.04597047, -0.03876879, ..., -0.00502494,
       -0.01078559, -0.0095915 ]), 'sampling_rate': 48000}, 'text': "OpenChat is a self play fine tune of the Mistral model. Notux-8x7B-v1. This is a version of Mixtral-8x7B, fine-tuned. Gemini Pro is Google's best model, or perhaps not as good as Gemini Ultra. Phi-2, I've already mentioned. DeciLM-7B is a high-speed 7B", 'start_time': '00:00:29.220', 'end_time': '00:00:58.240'}


In [10]:
from datasets import Audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
print(dataset["train"][0])

{'audio': {'path': 'segment_0.mp3', 'array': array([ 5.68434189e-14,  3.73034936e-14,  2.48689958e-14, ...,
       -4.04646471e-02, -3.87329012e-02, -4.26897369e-02]), 'sampling_rate': 16000}, 'text': "Spin. SPIN is self-play fine-tuning that improves LLMs. Tricksy, it's a form of fast inference involving sparsity. Phi-2 is a model from Microsoft. Lightning Attention 2 is an alternative to Flash Attention. Mixtral 8x7B, this is a Mixture of Experts model. Solar 10.7B is a Mistral model with some extra layers added in.", 'start_time': '00:00:00.000', 'end_time': '00:00:29.220'}


In [12]:
def prepare_dataset(batch):
  audio = batch["audio"]

  batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

  batch["labels"] = tokenizer(batch["text"]).input_ids

  return batch

In [13]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"][0:4])

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [14]:
dataset["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 6
})

In [15]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5
    })
})


In [16]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
  processor: Any

  def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
    input_features = [{"input_features": feature["input_features"]} for feature in features]
    batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

    label_features = [{"input_ids": feature["labels"]} for feature in features]
    labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
      labels = labels[:, 1:]

    batch["labels"] = labels

    return batch

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [18]:
import evaluate

metric = evaluate.load("wer")

In [19]:
def compute_metrics(pred):
  pred_ids = pred.predictions
  label_ids = pred.label_ids

  label_ids[label_ids == -100] = tokenizer.pad_token_id

  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

  wer = 100 * metric.compute(predictions=pred_str, references=label_str)

  return {"wer": wer}

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=False, device_map="auto")


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [21]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [22]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.9 MB/s eta 0:00:00


In [23]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.4429


In [24]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, TrainerCallback
from safetensors.torch import save_model
import os

In [25]:
class SafetensorsSaveCallback(TrainerCallback):
    def __init__(self, save_dir):
        self.save_dir = save_dir

    def on_save(self, args, state, control, **kwargs):
        if state.is_world_process_zero:
            model = kwargs['model']
            epoch = state.epoch
            save_path = os.path.join(self.save_dir, f"model_epoch_{epoch:.2f}.safetensors")
            save_model(model, save_path)
            print(f"Model saved using safetensors at {save_path}")
        return control

    def on_train_end(self, args, state, control, **kwargs):
        if state.is_world_process_zero:
            model = kwargs['model']
            save_path = os.path.join(self.save_dir, "model.safetensors")
            save_model(model, save_path)
            print(f"Final model saved using safetensors at {save_path}")
        return control

In [26]:
safetensors_callback = SafetensorsSaveCallback(save_dir=trained_model_repo)

In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=trained_model_repo,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,
    num_train_epochs=5,
    eval_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=1,
    generation_max_length=128,
    logging_steps=1,
    remove_unused_columns=False,
    label_names=["labels"],
    predict_with_generate=True,
    save_strategy="no",  # Disable default PyTorch saving
    push_to_hub=False,  # We'll handle this manually after safetensors saving
)

In [28]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor.feature_extractor,
    callbacks=[safetensors_callback],
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [29]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Wer
1,1.508400,1.789603,37.850467
2,0.988200,1.618672,42.056075
3,0.646400,1.527344,58.411215
4,0.429700,1.526768,39.252336
5,0.254100,1.534120,37.850467


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.proce

Final model saved using safetensors at johnkidsm/whisper-small-llm-ling/model.safetensors


TrainOutput(global_step=10, training_loss=0.7581813305616378, metrics={'train_runtime': 77.9291, 'train_samples_per_second': 0.385, 'train_steps_per_second': 0.128, 'total_flos': 8810444390400000.0, 'train_loss': 0.7581813305616378, 'epoch': 5.0})

In [31]:
model = model.merge_and_unload()

In [34]:
model.save_pretrained("./model")

In [35]:
processor.save_pretrained("./model")

[]

In [59]:
from transformers import pipeline

whisper_finetuned_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-small",
    #model="./model",
    chunk_length_s=30,
    device="cuda",
)

Device set to use cuda


In [60]:
def process_audio_and_create_vtt(audio_filename, audio_type, whisper_asr, output_filename):
  prediction = whisper_asr(f"{audio_filename}.{audio_type}", return_timestamps=True)

  vtt_file_name = output_filename if output_filename else f"{audio_filename}.vtt"

  with open(vtt_file_name, "w", encoding='utf-8') as vtt_file:
      vtt_file.write("WEBVTT\n\n")

      for i, chunk in enumerate(prediction.get("chunks", [])):
        start, end = chunk.get("timestamp", (None, None))
        text = chunk.get("text", "").strip()

        start_time = "1"
        end_time = "2"

        vtt_file.write(f"{start_time} --> {end_time}\n{text}\n\n")



In [61]:
process_audio_and_create_vtt("validation", "mp3", whisper_finetuned_asr, "eval.vtt")

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
